# scIDiff Basic Usage Tutorial

This notebook demonstrates the basic usage of scIDiff for single-cell gene expression modeling and inverse design.

## Overview

scIDiff is a deep generative framework that uses diffusion models to:
- Generate realistic single-cell expression profiles
- Denoise scRNA-seq data
- Predict perturbation responses
- Perform inverse design of cellular states

In [ ]:
# Import necessary libraries
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import scIDiff components
from scIDiff.models import ScIDiffModel
from scIDiff.training import ScIDiffTrainer
from scIDiff.sampling import InverseDesigner, PhenotypeTarget
from scIDiff.sampling.inverse_design import GeneExpressionObjective
from scIDiff.data import SingleCellDataset

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Data Preparation

First, let's create some synthetic single-cell data for demonstration purposes.

In [ ]:
# Create synthetic single-cell data
def create_synthetic_data(n_cells=1000, n_genes=2000, n_cell_types=5):
    """
    Create synthetic single-cell RNA-seq data
    """
    # Generate cell type labels
    cell_types = np.random.randint(0, n_cell_types, n_cells)
    
    # Generate gene expression data with cell type specific patterns
    expression_data = []
    
    for cell_type in range(n_cell_types):
        n_cells_type = np.sum(cell_types == cell_type)
        
        # Create cell type specific expression pattern
        base_expression = np.random.lognormal(0, 1, (n_cells_type, n_genes))
        
        # Add cell type specific markers
        marker_genes = np.random.choice(n_genes, 50, replace=False)
        base_expression[:, marker_genes] *= (cell_type + 1) * 2
        
        # Add sparsity (many genes are not expressed)
        sparsity_mask = np.random.random((n_cells_type, n_genes)) < 0.7
        base_expression[sparsity_mask] = 0
        
        expression_data.append(base_expression)
    
    # Combine all cell types
    expression_matrix = np.vstack(expression_data)
    
    # Log-normalize
    expression_matrix = np.log1p(expression_matrix)
    
    return expression_matrix, cell_types

# Generate synthetic data
expression_data, cell_type_labels = create_synthetic_data()

print(f"Expression data shape: {expression_data.shape}")
print(f"Cell types: {np.unique(cell_type_labels)}")
print(f"Sparsity: {(expression_data == 0).mean():.2%}")

In [ ]:
# Visualize the data
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Expression distribution
axes[0].hist(expression_data.flatten(), bins=50, alpha=0.7)
axes[0].set_xlabel('Log Expression')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Expression Distribution')

# Cell type distribution
unique, counts = np.unique(cell_type_labels, return_counts=True)
axes[1].bar(unique, counts)
axes[1].set_xlabel('Cell Type')
axes[1].set_ylabel('Number of Cells')
axes[1].set_title('Cell Type Distribution')

# Sparsity per cell
sparsity_per_cell = (expression_data == 0).mean(axis=1)
axes[2].hist(sparsity_per_cell, bins=30, alpha=0.7)
axes[2].set_xlabel('Sparsity (fraction of zeros)')
axes[2].set_ylabel('Number of Cells')
axes[2].set_title('Sparsity Distribution')

plt.tight_layout()
plt.show()

## 2. Model Initialization

Now let's initialize the scIDiff model with appropriate parameters.

In [ ]:
# Model configuration
model_config = {
    'gene_dim': expression_data.shape[1],  # Number of genes
    'hidden_dim': 512,                     # Hidden dimension
    'num_layers': 6,                       # Number of layers
    'num_timesteps': 1000,                 # Diffusion timesteps
    'conditioning_dim': 128,               # Conditioning dimension
    'dropout': 0.1,                        # Dropout rate
    'use_attention': True,                 # Use attention layers
}

# Initialize model
model = ScIDiffModel(**model_config)
model = model.to(device)

print(f"Model initialized with {sum(p.numel() for p in model.parameters()):,} parameters")
print(f"Model device: {next(model.parameters()).device}")

## 3. Data Loading

Prepare the data for training using PyTorch DataLoader.

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Convert to tensors
expression_tensor = torch.FloatTensor(expression_data)
cell_type_tensor = torch.LongTensor(cell_type_labels)

# Create dataset
class SimpleScDataset(torch.utils.data.Dataset):
    def __init__(self, expression, cell_types):
        self.expression = expression
        self.cell_types = cell_types
        
    def __len__(self):
        return len(self.expression)
    
    def __getitem__(self, idx):
        return {
            'expression': self.expression[idx],
            'conditioning': {
                'cell_type': self.cell_types[idx]
            }
        }

# Split data
train_size = int(0.8 * len(expression_tensor))
val_size = len(expression_tensor) - train_size

train_expression = expression_tensor[:train_size]
train_cell_types = cell_type_tensor[:train_size]
val_expression = expression_tensor[train_size:]
val_cell_types = cell_type_tensor[train_size:]

# Create datasets
train_dataset = SimpleScDataset(train_expression, train_cell_types)
val_dataset = SimpleScDataset(val_expression, val_cell_types)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Batch size: {train_loader.batch_size}")

## 4. Model Training

Train the scIDiff model using the prepared data.

In [ ]:
# Initialize trainer
trainer = ScIDiffTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    log_dir='./logs',
    checkpoint_dir='./checkpoints',
    use_wandb=False,  # Set to True if you want to use Weights & Biases
    gradient_clip_val=1.0
)

print("Trainer initialized successfully")
print(trainer.get_model_summary())

In [ ]:
# Train the model (reduced epochs for demo)
trainer.train(
    num_epochs=10,  # Use more epochs for real training
    save_every=5,
    validate_every=1,
    log_every=1
)

print("Training completed!")

## 5. Sample Generation

Generate new single-cell expression profiles using the trained model.

In [ ]:
# Generate unconditional samples
model.eval()
with torch.no_grad():
    generated_samples = model.sample(batch_size=100)
    generated_samples = generated_samples.cpu().numpy()

print(f"Generated samples shape: {generated_samples.shape}")
print(f"Generated samples sparsity: {(generated_samples == 0).mean():.2%}")

In [ ]:
# Compare real vs generated data
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Expression distributions
axes[0, 0].hist(expression_data.flatten(), bins=50, alpha=0.7, label='Real', density=True)
axes[0, 0].hist(generated_samples.flatten(), bins=50, alpha=0.7, label='Generated', density=True)
axes[0, 0].set_xlabel('Log Expression')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Expression Distribution Comparison')
axes[0, 0].legend()

# Mean expression per gene
real_mean = expression_data.mean(axis=0)
gen_mean = generated_samples.mean(axis=0)
axes[0, 1].scatter(real_mean, gen_mean, alpha=0.5)
axes[0, 1].plot([0, real_mean.max()], [0, real_mean.max()], 'r--')
axes[0, 1].set_xlabel('Real Mean Expression')
axes[0, 1].set_ylabel('Generated Mean Expression')
axes[0, 1].set_title('Mean Expression Correlation')

# Variance comparison
real_var = expression_data.var(axis=0)
gen_var = generated_samples.var(axis=0)
axes[1, 0].scatter(real_var, gen_var, alpha=0.5)
axes[1, 0].plot([0, real_var.max()], [0, real_var.max()], 'r--')
axes[1, 0].set_xlabel('Real Variance')
axes[1, 0].set_ylabel('Generated Variance')
axes[1, 0].set_title('Variance Correlation')

# Sparsity comparison
real_sparsity = (expression_data == 0).mean(axis=0)
gen_sparsity = (generated_samples == 0).mean(axis=0)
axes[1, 1].scatter(real_sparsity, gen_sparsity, alpha=0.5)
axes[1, 1].plot([0, 1], [0, 1], 'r--')
axes[1, 1].set_xlabel('Real Sparsity')
axes[1, 1].set_ylabel('Generated Sparsity')
axes[1, 1].set_title('Sparsity Correlation')

plt.tight_layout()
plt.show()

## 6. Conditional Generation

Generate samples conditioned on specific cell types.

In [ ]:
# Generate samples for each cell type
cell_type_samples = {}

for cell_type in range(5):  # 5 cell types
    conditioning = {
        'cell_type': torch.tensor([cell_type] * 50, device=device)
    }
    
    with torch.no_grad():
        samples = model.sample(batch_size=50, conditioning=conditioning)
        cell_type_samples[cell_type] = samples.cpu().numpy()

print("Generated conditional samples for all cell types")

In [ ]:
# Visualize cell type specific generation
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Mean expression per cell type
cell_type_means = []
for ct in range(5):
    real_ct_data = expression_data[cell_type_labels == ct]
    gen_ct_data = cell_type_samples[ct]
    
    real_mean = real_ct_data.mean()
    gen_mean = gen_ct_data.mean()
    
    cell_type_means.append([real_mean, gen_mean])

cell_type_means = np.array(cell_type_means)

x = np.arange(5)
width = 0.35

axes[0].bar(x - width/2, cell_type_means[:, 0], width, label='Real', alpha=0.7)
axes[0].bar(x + width/2, cell_type_means[:, 1], width, label='Generated', alpha=0.7)
axes[0].set_xlabel('Cell Type')
axes[0].set_ylabel('Mean Expression')
axes[0].set_title('Mean Expression by Cell Type')
axes[0].legend()
axes[0].set_xticks(x)

# Expression correlation between real and generated for each cell type
correlations = []
for ct in range(5):
    real_ct_data = expression_data[cell_type_labels == ct]
    gen_ct_data = cell_type_samples[ct]
    
    real_mean_genes = real_ct_data.mean(axis=0)
    gen_mean_genes = gen_ct_data.mean(axis=0)
    
    corr = np.corrcoef(real_mean_genes, gen_mean_genes)[0, 1]
    correlations.append(corr)

axes[1].bar(range(5), correlations, alpha=0.7)
axes[1].set_xlabel('Cell Type')
axes[1].set_ylabel('Gene Expression Correlation')
axes[1].set_title('Real vs Generated Correlation by Cell Type')
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

print(f"Average correlation across cell types: {np.mean(correlations):.3f}")

## 7. Inverse Design

Now let's demonstrate the inverse design capability - generating cells with specific gene expression targets.

In [ ]:
# Create gene name to index mapping (for demo purposes)
gene_names = [f'Gene_{i}' for i in range(expression_data.shape[1])]
gene_to_idx = {name: idx for idx, name in enumerate(gene_names)}

# Add some "marker" genes
marker_genes = ['Gene_0', 'Gene_1', 'Gene_2', 'Gene_10', 'Gene_20']
suppressed_genes = ['Gene_100', 'Gene_200', 'Gene_300']

print(f"Marker genes: {marker_genes}")
print(f"Suppressed genes: {suppressed_genes}")

In [ ]:
# Setup inverse design
objective_function = GeneExpressionObjective(gene_to_idx)
designer = InverseDesigner(
    model=model,
    objective_functions=[objective_function],
    device=device
)

# Define target phenotype
target_phenotype = PhenotypeTarget(
    gene_targets={
        'Gene_0': 5.0,   # High expression
        'Gene_1': 4.0,   # High expression
        'Gene_2': 3.0,   # Medium-high expression
    },
    marker_genes=marker_genes,
    suppressed_genes=suppressed_genes,
    cell_type='custom'
)

print("Target phenotype defined")
print(f"Gene targets: {target_phenotype.gene_targets}")

In [ ]:
# Perform inverse design
designed_cells = designer.design(
    target=target_phenotype,
    num_samples=32,
    num_optimization_steps=50,  # Reduced for demo
    learning_rate=0.01
)

designed_cells_np = designed_cells.cpu().numpy()
print(f"Designed cells shape: {designed_cells_np.shape}")

In [ ]:
# Evaluate inverse design results
evaluation_metrics = designer.evaluate_design(designed_cells, target_phenotype)

print("Inverse Design Evaluation:")
for metric, value in evaluation_metrics.items():
    print(f"  {metric}: {value:.4f}")

In [ ]:
# Visualize inverse design results
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Target gene expression levels
target_genes = list(target_phenotype.gene_targets.keys())
target_indices = [gene_to_idx[gene] for gene in target_genes]
target_values = list(target_phenotype.gene_targets.values())
achieved_values = designed_cells_np[:, target_indices].mean(axis=0)

x = np.arange(len(target_genes))
width = 0.35

axes[0, 0].bar(x - width/2, target_values, width, label='Target', alpha=0.7)
axes[0, 0].bar(x + width/2, achieved_values, width, label='Achieved', alpha=0.7)
axes[0, 0].set_xlabel('Target Genes')
axes[0, 0].set_ylabel('Expression Level')
axes[0, 0].set_title('Target vs Achieved Expression')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(target_genes, rotation=45)
axes[0, 0].legend()

# Marker genes expression
marker_indices = [gene_to_idx[gene] for gene in marker_genes]
marker_expression = designed_cells_np[:, marker_indices]
random_expression = np.random.choice(designed_cells_np.flatten(), size=marker_expression.shape)

axes[0, 1].boxplot([marker_expression.flatten(), random_expression.flatten()], 
                   labels=['Marker Genes', 'Random Genes'])
axes[0, 1].set_ylabel('Expression Level')
axes[0, 1].set_title('Marker Genes vs Random Genes')

# Suppressed genes expression
suppressed_indices = [gene_to_idx[gene] for gene in suppressed_genes]
suppressed_expression = designed_cells_np[:, suppressed_indices]

axes[1, 0].boxplot([suppressed_expression.flatten(), random_expression.flatten()], 
                   labels=['Suppressed Genes', 'Random Genes'])
axes[1, 0].set_ylabel('Expression Level')
axes[1, 0].set_title('Suppressed Genes vs Random Genes')

# Overall expression distribution
axes[1, 1].hist(expression_data.flatten(), bins=50, alpha=0.5, label='Original Data', density=True)
axes[1, 1].hist(designed_cells_np.flatten(), bins=50, alpha=0.5, label='Designed Cells', density=True)
axes[1, 1].set_xlabel('Expression Level')
axes[1, 1].set_ylabel('Density')
axes[1, 1].set_title('Expression Distribution Comparison')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 8. Model Evaluation

Let's evaluate the model's performance using various metrics.

In [ ]:
# Calculate evaluation metrics
def calculate_metrics(real_data, generated_data):
    """
    Calculate various evaluation metrics
    """
    metrics = {}
    
    # Mean and variance correlation
    real_mean = real_data.mean(axis=0)
    gen_mean = generated_data.mean(axis=0)
    metrics['mean_correlation'] = np.corrcoef(real_mean, gen_mean)[0, 1]
    
    real_var = real_data.var(axis=0)
    gen_var = generated_data.var(axis=0)
    metrics['variance_correlation'] = np.corrcoef(real_var, gen_var)[0, 1]
    
    # Sparsity comparison
    real_sparsity = (real_data == 0).mean()
    gen_sparsity = (generated_data == 0).mean()
    metrics['sparsity_difference'] = abs(real_sparsity - gen_sparsity)
    
    # Distribution similarity (using Wasserstein distance)
    from scipy.stats import wasserstein_distance
    metrics['wasserstein_distance'] = wasserstein_distance(
        real_data.flatten(), generated_data.flatten()
    )
    
    return metrics

# Evaluate unconditional generation
unconditional_metrics = calculate_metrics(expression_data, generated_samples)

print("Unconditional Generation Metrics:")
for metric, value in unconditional_metrics.items():
    print(f"  {metric}: {value:.4f}")

## 9. Save and Load Model

Demonstrate how to save and load the trained model.

In [ ]:
# Save the model
model_save_path = 'scidiff_demo_model.pt'

torch.save({
    'model_state_dict': model.state_dict(),
    'model_config': model_config,
    'training_metrics': unconditional_metrics
}, model_save_path)

print(f"Model saved to {model_save_path}")

In [ ]:
# Load the model
checkpoint = torch.load(model_save_path, map_location=device)

# Create new model instance
loaded_model = ScIDiffModel(**checkpoint['model_config'])
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_model = loaded_model.to(device)
loaded_model.eval()

print("Model loaded successfully")
print(f"Loaded model has {sum(p.numel() for p in loaded_model.parameters()):,} parameters")

In [ ]:
# Test loaded model
with torch.no_grad():
    test_samples = loaded_model.sample(batch_size=10)
    print(f"Generated test samples shape: {test_samples.shape}")
    print("Model loading and testing successful!")

## 10. Summary

In this tutorial, we demonstrated:

1. **Data Preparation**: Creating synthetic single-cell data
2. **Model Initialization**: Setting up the scIDiff model
3. **Training**: Training the diffusion model on single-cell data
4. **Generation**: Generating new single-cell expression profiles
5. **Conditional Generation**: Generating samples conditioned on cell types
6. **Inverse Design**: Designing cells with specific gene expression targets
7. **Evaluation**: Assessing model performance with various metrics
8. **Model Persistence**: Saving and loading trained models

### Next Steps

- Try with real single-cell datasets
- Experiment with different model architectures
- Implement custom objective functions for inverse design
- Explore perturbation prediction capabilities
- Scale up training with larger datasets and more epochs

### Key Takeaways

- scIDiff provides a flexible framework for single-cell generative modeling
- The inverse design capability enables targeted cellular state generation
- The model can be conditioned on various biological covariates
- Proper evaluation is crucial for assessing generation quality